In [321]:
import string
from collections import Counter
import re
import random 
from copy import copy


import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk import everygrams
from tqdm.notebook import tqdm

np.random.seed(42)

## Задание 1

Реализуйте базовый частотный метод по Шерлоку Холмсу:
   - подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
   - возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
   - расшифруйте их таким частотным методом.



In [322]:
tokenizer = RegexpTokenizer(r"\w+")
alphabet_ru = " абвгдежзийклмнопрстуфхцчшщъыьэюя"
alphabet_eng = " " + string.ascii_lowercase

In [323]:
def tokenize(text, tokenizer=tokenizer, alphabet=alphabet_ru):   
    return " ".join(tokenizer.tokenize("".join([char for char in text if char.lower() in alphabet]).lower()))

In [324]:
def get_frequencies(text, min_freq=0, n_gram=1):
    frequencies = dict()
    if n_gram > 1:
        text = ["".join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)]
    for key, value in Counter(text).items():
        if value / len(text) > min_freq:
            frequencies[key] = value / len(text)
    
    return frequencies

In [325]:
def generate_random_mapping(frequencies):
    chars = list(frequencies.keys())
    replace_chars = np.random.choice(chars, replace=False, size=len(frequencies))
    mapping = dict()
    for original_char, replace_char in zip(chars, replace_chars):
        mapping[original_char] = replace_char
    
    return mapping

In [326]:
def apply_mapping(text, mapping):

    return "".join([mapping.get(char, "ь") for char in text])


In [327]:
def generate_reverse_mapping(corpus_frequencies, text_frequencies):
    corpus_frequencies_sorted = sorted(corpus_frequencies.items(), key=lambda x: x[1], reverse=True)
    text_frequencies_sorted = sorted(text_frequencies.items(), key=lambda x: x[1], reverse=True)
    
    reverse_mapping = dict()
    for text_char, text_freq in text_frequencies_sorted:
        min_diff = 1
        best_char = None
        for corpus_char, corpus_freq in corpus_frequencies_sorted:
            diff = abs(corpus_freq - text_freq)
            if diff < min_diff:
                best_char = corpus_char
                min_diff = diff
        
        reverse_mapping[text_char] = best_char
        corpus_freqeuncies_sorted = [(char, freq) for char, freq in corpus_frequencies_sorted if char != best_char]
        
    return reverse_mapping

In [328]:
# score function
def char_accuracy(text_1, text_2):
    assert len(text_1) == len(text_2)
    right_chars = 0
    for c1, c2 in zip(text_1, text_2):
        right_chars += int(c1 == c2)
        
    return right_chars / len(text_1)

In [329]:
# score function
def levenstein_distance(text_1, text_2):
    assert len(text_1) == len(text_2)
    distances = range(len(text_1) + 1)
    for idx_2, char_2 in enumerate(text_2):
        new_dist = [idx_2 + 1]
        for idx_1, char_1 in enumerate(text_1):
            if char_1 == char_2:
                new_dist.append(distances[idx_1])
            else:
                new_dist.append(1 + min(distances[idx_1], distances[idx_1 + 1], new_dist[-1]))
        distances = new_dist

    return distances[-1]

In [330]:
# read data

with open("AnnaKarenina.txt", "r", encoding="utf8") as file:
    corpus_1 = file.readlines()
    
with open("WarAndPeace.txt", "r", encoding="utf8") as file:
    corpus_2 = file.readlines()
    
corpus = corpus_1 + corpus_2

tokenized_corpus = tokenize(" ".join(corpus))
corpus_frequencies = get_frequencies(tokenized_corpus)
mapping = generate_random_mapping(corpus_frequencies)

In [331]:
with open("Tru.txt", "r", encoding="utf8") as file:
    text = file.readlines()
    
tokenized_test = tokenize(" ".join(text))
encoded_test = apply_mapping(tokenized_test, mapping)
test_frequencies = get_frequencies(encoded_test)
tokenized_test

'наташа вышла замуж ранней весной года а в году уже родила трех дочерей не считая сына которого она так долго ждала и которого теперь кормила она стала полнее и шире так что в этой крепкой материнской женщине трудно было узнать стройную живую наташу прежних дней ее черты лица были более четкими и имели спокойное мягкое и безмятежное выражение на ее лице не было ничего из вечно светящегося оживления которое раньше горело здесь и составляло его очарование теперь ее лицо и тело часто были всем что можно было увидеть а ее души не было видно совсем все что бросалось в глаза это сильная красивая и плодородная женщина старый огонь теперь очень редко загорался перед ее лицом это произошло только тогда когда как это было в тот день ее муж вернулся домой или больной ребенок выздоравливал или когда она и графиня марья заговорили о князе эндрю она никогда не упоминала о нем своему мужу который как ей казалось ревновал князя эндрю или в тех редких случаях когда чтото заставляло ее петь практика от 

In [332]:
encoded_test

'вщмщфщупефнщуащлижудщввзбупзрвшбуюшыщущупуюшыиуижзудшычнщумдзхуышкздзбувзуркчмщэуревщуьшмшдшюшушвщумщьуышнюшужыщнщучуьшмшдшюшумзгзд уьшдлчнщушвщурмщнщугшнвззучуфчдзумщьукмшупуямшбуьдзгьшбулщмздчврьшбужзвтчвзумдиывшусеншуиавщм урмдшбвиоужчпиоувщмщфиугдзжвчхуывзбуззукздмеунчъщусенчусшнззукзмьчлчучучлзнчургшьшбвшзулэюьшзучусзалэмзжвшзупедщжзвчзувщуззунчъзувзусеншувчкзюшучаупзквшурпзмэтзюшрэушжчпнзвчэуьшмшдшзудщв фзуюшдзншуаызр учуршрмщпнэншузюшушкщдшпщвчзумзгзд уззунчъшучумзншукщрмшусенчупрзлукмшулшжвшусеншуипчызм ущуззуыифчувзусеншупчывшуршпрзлупрзукмшусдшрщншр упуюнщащуямшурчн вщэуьдщрчпщэучугншышдшывщэужзвтчвщурмщдебушюшв умзгзд ушкзв удзыьшуащюшдщнрэугздзыуззунчъшлуямшугдшчашфншумшн ьшумшюыщуьшюыщуьщьуямшусеншупумшмуызв уззулижупздвинрэуышлшбучнчусшн вшбудзсзвшьупеаышдщпнчпщнучнчуьшюыщушвщучуюдщйчвэулщд эуащюшпшдчнчушуьвэазуявыдоушвщувчьшюыщувзуигшлчвщнщушувзлурпшзлиулижиуьшмшдебуьщьузбуьщащншр удзпвшпщнуьвэаэуявыдоучнчупумзхудзыьчхурникщэхуьшюыщукмшмшуащрмщпнэншуззугзм угдщьмчьщушму

In [333]:
reverse_mapping = generate_reverse_mapping(corpus_frequencies, test_frequencies)
decoded_test = apply_mapping(encoded_test, reverse_mapping)
decoded_test

'татаюа кбюса ыапыч латтоб коктоб яода а к яоды ычо лодтса тлою дочолоб то кчттаь кбта котолояо ота так досяо чдаса т котолояо тоболя колптса ота ктаса бостоо т ютло так что к ютоб клобкоб патолттккоб чотэтто тлыдто жбсо ыытатя ктлобтыю чткыю татаюы блочттю дтоб оо чолтб стэа жбст жосоо чотктпт т тпост кбокобтоо пьякоо т жоыпьточтоо кблачотто та оо стэо то жбсо ттчояо ты кочто ккотьэояокь очтксотть котолоо латяюо яолосо ыдокя т коктаксьсо ояо очалокатто тоболя оо стэо т тосо чакто жбст ккоп что почто жбсо ыктдотя а оо дыют то жбсо ктдто коккоп кко что жлокасокя к ясаыа юто ктсятаь клакткаь т бсодолодтаь чотэтта кталбб ояотя тоболя очотя лодко ыаяоласкь болод оо стэоп юто блотыоюсо тосяко тояда кояда как юто жбсо к тот дотя оо пыч колтыскь допоб тст жосятоб ложоток кбыдолаксткас тст кояда ота т ялафтть паляь ыаяоколтст о ктьыо ютдлю ота тткояда то ыбопттаса о топ ккоопы пычы котолбб как об каыасокя локтокас ктьыь ютдлю тст к тою лодктю ксычаью кояда чтото ыактаксьсо оо ботя блакттка от 

In [334]:
print(f"Char decoding accuracy: {char_accuracy(tokenized_test, decoded_test) * 100:.2f}%")
print(f"Levenstein distace: {levenstein_distance(tokenized_test, decoded_test)}")

Char decoding accuracy: 46.34%
Levenstein distace: 637


Плохая расшифровка.

А что по поводу английских текстов?

In [335]:
with open("WarAndPeaceEng.txt", "r", encoding="utf8") as file:
    eng_text = file.readlines()

tokenized_corpus_eng = tokenize(" ".join(eng_text), tokenizer=tokenizer, alphabet=alphabet_eng)
corpus_frequencies_eng = get_frequencies(tokenized_corpus_eng)
mapping_eng = generate_random_mapping(corpus_frequencies_eng)

In [336]:
with open("Teng.txt", "r", encoding="utf8") as file:
    text = file.readlines()
    
tokenized_test = tokenize(" ".join(text), tokenizer=tokenizer, alphabet=alphabet_eng)
encoded_test = apply_mapping(tokenized_test, mapping_eng)
test_frequencies_eng = get_frequencies(encoded_test)
tokenized_test

'natasha had married in the early spring of and in already had three daughters besides a son for whom she had longed and whom she was now nursing she had grown stouter and broader so that it was difficult to recognize in this robust motherly woman the slim lively natasha of former days her features were more defined and had a calm soft and serene expression in her face there was none of the everglowing animation that had formerly burned there and constituted its charm now her face and body were often all that one saw and her soul was not visible at all all that struck the eye was a strong handsome and fertile woman the old fire very rarely kindled in her face now that happened only when as was the case that day her husband returned home or a sick child was convalescent or when she and countess mary spoke of prince andrew she never mentioned him to her husband who she imagined was jealous of prince andrews memory or on the rare occasions when something happened to induce her to sing a p

In [337]:
encoded_test

'ictclrczrcozycjjnsozniztrszscjepzlkjniqzmhzcioznizcejscopzrcoztrjsszocvqrtsjlzaslnoslzczlmizhmjzdrmyzlrszrcozemiqsozciozdrmyzlrszdclzimdzivjlniqzlrszrcozqjmdizltmvtsjzciozajmcosjzlmztrctzntzdclzonhhnbvetztmzjsbmqinwszniztrnlzjmavltzymtrsjepzdmyciztrszlenyzenxsepzictclrczmhzhmjysjzocplzrsjzhsctvjslzdsjszymjszoshnisozciozrcozczbceyzlmhtzciozlsjsiszsfkjsllnmiznizrsjzhcbsztrsjszdclzimiszmhztrszsxsjqemdniqzcinyctnmiztrctzrcozhmjysjepzavjisoztrsjszciozbmiltntvtsozntlzbrcjyzimdzrsjzhcbszciozamopzdsjszmhtsizceeztrctzmiszlcdzciozrsjzlmvezdclzimtzxnlnaeszctzceezceeztrctzltjvb ztrszspszdclzczltjmiqzrciolmyszciozhsjtneszdmyciztrszmeozhnjszxsjpzjcjsepz nioesoznizrsjzhcbszimdztrctzrckksisozmiepzdrsizclzdclztrszbclsztrctzocpzrsjzrvlaciozjstvjisozrmyszmjzczlnb zbrneozdclzbmixceslbsitzmjzdrsizlrszciozbmvitsllzycjpzlkm szmhzkjnibszciojsdzlrszisxsjzysitnmisozrnyztmzrsjzrvlaciozdrmzlrsznycqnisozdclzuscemvlzmhzkjnibszciojsdlzysymjpzmjzmiztrszjcjszmbbclnmilzdrsizlmystrniqzrckksisoztmzniovbszrsjztmzlniqzczk

In [338]:
reverse_mapping = generate_reverse_mapping(corpus_frequencies_eng, test_frequencies_eng)
decoded_test = apply_mapping(encoded_test, reverse_mapping)
decoded_test

'athtsit itr utnnder da hie etnlp svndab iw tar da tlnetrp itr hinee rtybihens vesdres t sia win uiiu sie itr liaber tar uiiu sie uts aiu aynsdab sie itr bniua shiyhen tar vnitren si hith dh uts rdwwdmylh hi nemibadje da hids nivysh uihienlp uiuta hie sldu ldkelp athtsit iw winuen rtps ien wethynes uene uine rewdaer tar itr t mtlu siwh tar seneae ejvnessdia da ien wtme hiene uts aiae iw hie ekenbliudab taduthdia hith itr winuenlp vynaer hiene tar miashdhyher dhs mitnu aiu ien wtme tar virp uene iwhea tll hith iae stu tar ien siyl uts aih kdsdvle th tll tll hith shnymk hie epe uts t shniab itarsiue tar wenhdle uiuta hie ilr wdne kenp ntnelp kdarler da ien wtme aiu hith itvveaer ialp uiea ts uts hie mtse hith rtp ien iysvtar nehynaer iiue in t sdmk midlr uts miaktlesmeah in uiea sie tar miyahess utnp svike iw vndame tarneu sie aeken ueahdiaer idu hi ien iysvtar uii sie dutbdaer uts jetliys iw vndame tarneus ueuinp in ia hie ntne immtsdias uiea siuehidab itvveaer hi daryme ien hi sdab t v

In [339]:
print(f"Char decoding accuracy: {char_accuracy(tokenized_test, decoded_test) * 100:.2f}%")
print(f"Levenstein distace: {levenstein_distance(tokenized_test, decoded_test)}")

Char decoding accuracy: 37.92%
Levenstein distace: 724


Был взят одинаковый текст на русском и английском, но на русском получилось лучше, поскольку букв в языке меньше

## Задание 2

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм


In [374]:
# read data

with open("AnnaKarenina.txt", "r", encoding="utf8") as file:
    corpus_1 = file.readlines()
    
with open("WarAndPeace.txt", "r", encoding="utf8") as file:
    corpus_2 = file.readlines()
    
corpus = corpus_1 + corpus_2

tokenized_corpus = tokenize(" ".join(corpus))
corpus_frequencies = get_frequencies(tokenized_corpus)
mapping = generate_random_mapping(corpus_frequencies)

In [375]:
with open("Tru.txt", "r", encoding="utf8") as file:
    text = file.readlines()
    
tokenized_test = tokenize(" ".join(text))
encoded_test = apply_mapping(tokenized_test, mapping)
tokenized_test

'наташа вышла замуж ранней весной года а в году уже родила трех дочерей не считая сына которого она так долго ждала и которого теперь кормила она стала полнее и шире так что в этой крепкой материнской женщине трудно было узнать стройную живую наташу прежних дней ее черты лица были более четкими и имели спокойное мягкое и безмятежное выражение на ее лице не было ничего из вечно светящегося оживления которое раньше горело здесь и составляло его очарование теперь ее лицо и тело часто были всем что можно было увидеть а ее души не было видно совсем все что бросалось в глаза это сильная красивая и плодородная женщина старый огонь теперь очень редко загорался перед ее лицом это произошло только тогда когда как это было в тот день ее муж вернулся домой или больной ребенок выздоравливал или когда она и графиня марья заговорили о князе эндрю она никогда не упоминала о нем своему мужу который как ей казалось ревновал князя эндрю или в тех редких случаях когда чтото заставляло ее петь практика от 

In [376]:
corpus_frequencies_bigram = get_frequencies(tokenized_corpus, n_gram=2)
test_frequencies_bigram = get_frequencies(encoded_test, n_gram=2)

In [377]:
def generate_reverse_mapping_bigram(corpus_frequencies, text_frequencies):
    corpus_frequencies_sorted = sorted(corpus_frequencies.items(), key=lambda x: x[1], reverse=True)
    text_frequencies_sorted = sorted(text_frequencies.items(), key=lambda x: x[1], reverse=True)
    
    reverse_mapping = dict()
    
    for i, (text_bigram, text_freq) in enumerate(text_frequencies_sorted):
        filtered_frequencies = copy(corpus_frequencies_sorted)
        
        if text_bigram[0] in reverse_mapping.keys():
            filtered_frequencies = [(bigram, freq) for bigram, freq in filtered_frequencies 
                                    if bigram[0] == reverse_mapping[text_bigram[0]]]
            
        if text_bigram[1] in reverse_mapping.keys():
            filtered_frequencies = [(bigram, freq) for bigram, freq in filtered_frequencies 
                                    if bigram[1] == reverse_mapping[text_bigram[1]]]
            
        min_diff = 1
        best_bigram = None
        for bigram, freq in filtered_frequencies:
            diff = abs(freq - text_freq)
            if diff < min_diff:
                best_bigram = bigram
                min_diff = diff
        
        if text_bigram[0] not in reverse_mapping.keys():
            reverse_mapping[text_bigram[0]] = best_bigram[0]
        
        if text_bigram[1] not in reverse_mapping.keys():
            reverse_mapping[text_bigram[1]] = best_bigram[1] 
    
    return reverse_mapping

In [378]:
reverse_mapping_bigram = generate_reverse_mapping_bigram(corpus_frequencies_bigram, test_frequencies_bigram)
decoded_test = apply_mapping(encoded_test, reverse_mapping_bigram)
decoded_test

' отожо пажно рочт  но  оя пои оя  оро о п  орт т о норяно тноы робоноя  о ибятоь иа о вотоно о о о тов рон о  роно я вотоно о токоня вончяно о о итоно кон оо я жяно тов бто п етоя вноквоя чотоня ивоя  о хя о тнтр о кано тр отя итноя ты  япты  отожт кно  яы р оя оо бонта нято каня коноо ботвячя я ячоня иковоя оо чь воо я корчьто  оо пано о яо  о оо нято  о кано  ябо о яр поб о ипотьхо оиь о япно яь вотоноо но яжо  ононо рроия я иоитопньно о о обонопо яо токоня оо нято я тоно боито каня пиоч бто чо  о кано тпяротя о оо ртжя  о кано пяр о иопиоч пио бто кноионоия п  норо ето ияня оь вноияпоь я кноронор оь  о хя о итоная о о я токоня обо я норво ро онониь конор оо няточ ето кноярожно тоняво то ро во ро вов ето кано п тот ро я оо чт  пон тниь рочоя яня коня оя ноко ов парронопняпон яня во ро о о я  ноюя ь чоняь ро опоняня о в ьро е рны о о  яво ро  о ткочя оно о  оч ипоочт чт т вотоная вов оя вороноия ноп опон в ьрь е рны яня п тоы норвяы интбоьы во ро бтото роитопньно оо котя кновтяво от 

In [379]:
print(f"Char decoding accuracy: {char_accuracy(tokenized_test, decoded_test) * 100:.2f}%")
print(f"Levenstein distace: {levenstein_distance(tokenized_test, decoded_test)}")

Char decoding accuracy: 31.84%
Levenstein distace: 808


Метод частотных биграмм пракстически не работает

## Задание 3

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.


**Метод:**<br>
Биграммы в тексте - звенья марковской цепи, а частота биграмм - вероятность перехода.
Для обучения считается вероятность порождение текста как произведение всех биграмм, входящих в него.


In [88]:
def get_frequencies(text, min_freq=0, n_gram=2):
    frequencies = dict()
    length = len(set(text)) ** n_gram # all possible n_grams
    
    text = ["".join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)]
    for key, value in Counter(text).items():
        frequencies[key] = (value + 1) / (len(text) + length) # to avoid absent n_grams
        
    return frequencies

In [89]:
def get_text_probability(text, mapping, frequencies, n_gram=2, alphabet=alphabet_ru):
    decoded_text = apply_mapping(text, mapping)
    
    log_probability = 0
    for i in range(len(decoded_text) - n_gram):
        bigram = decoded_text[i: i + n_gram]
        absent_bigram_probability = 1 / (len(text) + len(alphabet) ** n_gram) # to avoid 0 probs
        bigram_probability = frequencies.get(bigram, absent_bigram_probability)
        
        log_probability += np.log(bigram_probability)
    
    return log_probability

In [90]:
def generate_mcmc_reverse_mapping(encoded_text, alphabet_encoded, alphabet_corpus, corpus_frequencies, 
                                  n_iters=10000, epochs=10, n_gram=2, verbose=True):
    
    accepted = 0
    best_mapping = None
    best_ll = -np.inf # log-likelihood for scoring
    
    for epoch in range(epochs):
        
        alphabet_encoded = list(alphabet_encoded)
        alphabet_iter = list(alphabet_corpus)
        reverse_mapping = {key: value for key, value 
                               in zip(alphabet_encoded, alphabet_iter[:len(alphabet_encoded)])}
        current_log_probability = get_text_probability(encoded_text, reverse_mapping, corpus_frequencies, n_gram=n_gram)
        
        for i in tqdm(range(n_iters), total=n_iters, leave=False, disable=not verbose):
            alphabet_proposal = alphabet_iter[:]
            idx_1, idx_2 = np.random.choice(len(alphabet_proposal), replace=False, size=2)
            alphabet_proposal[idx_1], alphabet_proposal[idx_2] = alphabet_proposal[idx_2], alphabet_proposal[idx_1]
            reverse_mapping_proposal = {key: value for key, value 
                               in zip(alphabet_encoded, alphabet_proposal[:len(alphabet_encoded)])}
            proposal_log_probability = get_text_probability(encoded_text, reverse_mapping_proposal, corpus_frequencies, n_gram=n_gram)
            
            p_accept = np.exp(proposal_log_probability - current_log_probability)
            
            if p_accept > np.random.rand():
                accepted += 1
                alphabet_iter = alphabet_proposal
                current_log_probability = proposal_log_probability
                reverse_mapping = reverse_mapping_proposal
                
        if current_log_probability > best_ll:
            best_ll = current_log_probability
            best_mapping = reverse_mapping
        print(f"Epoch #{epoch + 1}")
        print(f"Current log-likelihood: {current_log_probability}")
        print(f"Current accepted proposals: {accepted}")
 
    print(f"Best likelihood: {best_ll}")
    print(f"Acceptance ratio: {accepted / (n_iters * epochs)}")

    return best_mapping

In [91]:
corpus_frequencies = get_frequencies(tokenized_corpus, n_gram=2)

In [48]:
EPOCHS = 10

In [49]:
reverse_mapping = generate_mcmc_reverse_mapping(encoded_test, alphabet_ru, alphabet_ru, corpus_frequencies, epochs=EPOCHS)

Epoch #1
Current log-likelihood: -6412.2681926598425
Current accepted proposals: 153


Epoch #2
Current log-likelihood: -7323.316534151016
Current accepted proposals: 282


Epoch #3
Current log-likelihood: -6411.860370552869
Current accepted proposals: 406


Epoch #4
Current log-likelihood: -6411.860370552869
Current accepted proposals: 518


Epoch #5
Current log-likelihood: -6411.860370552869
Current accepted proposals: 642


Epoch #6
Current log-likelihood: -7491.35476413049
Current accepted proposals: 752


Epoch #7
Current log-likelihood: -6414.708294190226
Current accepted proposals: 894


Epoch #8
Current log-likelihood: -6411.860370552869
Current accepted proposals: 1032


Epoch #9
Current log-likelihood: -6412.2681926598425
Current accepted proposals: 1165


Epoch #10
Current log-likelihood: -7296.117600167826
Current accepted proposals: 1283
Best likelihood: -6411.860370552869
Acceptance ratio: 0.01283


In [50]:
decoded_test = apply_mapping(encoded_test, reverse_mapping)

In [51]:
decoded_test

'наташа вышла замуж ранней весной года а в году уже родила трех дочерей не считая сына которого она так долго ждала и которого теперь кормила она стала полнее и шире так что в этой крепкой материнской женщине трудно было узнать стройную живую наташу прежних дней ее черты лица были более четкими и имели спокойное мягкое и безмятежное выражение на ее лице не было ничего из вечно светящегося оживления которое раньше горело здесь и составляло его очарование теперь ее лицо и тело часто были всем что можно было увидеть а ее души не было видно совсем все что бросалось в глаза это сильная красивая и плодородная женщина старый огонь теперь очень редко загорался перед ее лицом это произошло только тогда когда как это было в тот день ее муж вернулся домой или больной ребенок выздоравливал или когда она и граъиня марья заговорили о князе эндрю она никогда не упоминала о нем своему мужу который как ей казалось ревновал князя эндрю или в тех редких случаях когда чтото заставляло ее петь практика от 

In [52]:
print(f"Char decoding accuracy: {char_accuracy(tokenized_test, decoded_test) * 100:.2f}%")
print(f"Levenstein distace: {levenstein_distance(tokenized_test, decoded_test)}")

Char decoding accuracy: 99.92%
Levenstein distace: 1


Текст расшифрован.

## Задание 4

Расшифруйте сообщение:

In [53]:
message_1 = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"
message_2 = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"

In [54]:
corpus_frequencies = get_frequencies(tokenized_corpus, n_gram=2)
message_frequencies = get_frequencies(message_1, n_gram=2)

In [55]:
corpus_frequencies_sorted = sorted(corpus_frequencies.items(), key=lambda x: x[1], reverse=True)
message_frequencies_sorted = sorted(message_frequencies.items(), key=lambda x: x[1], reverse=True)

In [56]:
alphabet_message = " " + "".join([char for char in set(message_1)])

In [61]:
EPOCHS = 100

In [62]:
reverse_mapping = generate_mcmc_reverse_mapping(message_1, alphabet_message, alphabet_ru, corpus_frequencies, epochs=EPOCHS)

Epoch #1
Current log-likelihood: -1377.802211286503
Current accepted proposals: 481


Epoch #2
Current log-likelihood: -1352.671765468478
Current accepted proposals: 942


Epoch #3
Current log-likelihood: -1361.47523638021
Current accepted proposals: 1446


Epoch #4
Current log-likelihood: -1261.3815000036611
Current accepted proposals: 1954


Epoch #5
Current log-likelihood: -1356.5826545325597
Current accepted proposals: 2463


...

Epoch #96
Current log-likelihood: -1258.799177990357
Current accepted proposals: 46723


Epoch #97
Current log-likelihood: -1379.0693932963252
Current accepted proposals: 47185


Epoch #98
Current log-likelihood: -1271.1129669705292
Current accepted proposals: 47701


Epoch #99
Current log-likelihood: -1262.9296795661298
Current accepted proposals: 48201


Epoch #100
Current log-likelihood: -1236.666079154639
Current accepted proposals: 48627
Best likelihood: -1231.2547551943435
Acceptance ratio: 0.048627


In [63]:
apply_mapping(message_1, reverse_mapping)

'если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешаъ'

In [64]:
message_answer = "если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю"
decoded_test = apply_mapping(message_1, reverse_mapping)

In [66]:
print(f"Char decoding accuracy: {char_accuracy(message_answer, decoded_test) * 100:.2f}%")
print(f"Levenstein distace: {levenstein_distance(message_answer, decoded_test)}")

Char decoding accuracy: 93.48%
Levenstein distace: 15


Сообщение почти расшифровано.


Для второго сообщения.

In [85]:
alphabet_message = " " + "".join([char for char in set(message_2)])

In [86]:
reverse_mapping = generate_mcmc_reverse_mapping(message_2, alphabet_message, alphabet_ru, corpus_frequencies, epochs=EPOCHS)

Epoch #1
Current log-likelihood: -1410.5614178687813
Current accepted proposals: 463


Epoch #2
Current log-likelihood: -1233.3178097020095
Current accepted proposals: 937


Epoch #3
Current log-likelihood: -1289.177646120717
Current accepted proposals: 1487


Epoch #4
Current log-likelihood: -1415.6123130256765
Current accepted proposals: 1961


Epoch #5
Current log-likelihood: -1368.1166650393443
Current accepted proposals: 2478


...

Epoch #96
Current log-likelihood: -1416.9103031241204
Current accepted proposals: 46911


Epoch #97
Current log-likelihood: -1322.9489407301464
Current accepted proposals: 47302


Epoch #98
Current log-likelihood: -1409.3376695294837
Current accepted proposals: 47814


Epoch #99
Current log-likelihood: -1384.0931484538037
Current accepted proposals: 48247


Epoch #100
Current log-likelihood: -1231.2547551943435
Current accepted proposals: 48728
Best likelihood: -1231.2547551943435
Acceptance ratio: 0.048728


In [87]:
decoded_test = apply_mapping(message_2, reverse_mapping)
print(f"Char decoding accuracy: {char_accuracy(message_answer, decoded_test) * 100:.2f}%")
print(f"Levenstein distace: {levenstein_distance(message_answer, decoded_test)}")
decoded_test

Char decoding accuracy: 93.48%
Levenstein distace: 15


'если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешаъ'

Работает аналогично и для второго сообщения.

## Бонус: Задание 5

 а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

In [70]:
message = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"
alphabet_message = " " + "".join([char for char in set(message)])

#### Триграммы

In [71]:
corpus_frequencies_trigramm = get_frequencies(tokenized_corpus, n_gram=3)

In [77]:
reverse_mapping = generate_mcmc_reverse_mapping(message, alphabet_message, alphabet_ru, corpus_frequencies_trigramm, n_gram=3, epochs=EPOCHS)

Epoch #1
Current log-likelihood: -1938.011392857163
Current accepted proposals: 468


Epoch #2
Current log-likelihood: -2094.0014443476825
Current accepted proposals: 1194


Epoch #3
Current log-likelihood: -2041.29797155999
Current accepted proposals: 1720


Epoch #4
Current log-likelihood: -1976.2782081022376
Current accepted proposals: 2287


Epoch #5
Current log-likelihood: -2041.2512037252773
Current accepted proposals: 2898


...

Epoch #96
Current log-likelihood: -2059.045874171524
Current accepted proposals: 57988


Epoch #97
Current log-likelihood: -1725.6602016838426
Current accepted proposals: 58437


Epoch #98
Current log-likelihood: -1959.2126550357964
Current accepted proposals: 59152


Epoch #99
Current log-likelihood: -2108.7117288524487
Current accepted proposals: 59910


Epoch #100
Current log-likelihood: -1725.6602016838426
Current accepted proposals: 60370
Best likelihood: -1725.6602016838426
Acceptance ratio: 0.06037


In [78]:
decoded_test = apply_mapping(message, reverse_mapping)
print(f"Char decoding accuracy: {char_accuracy(message_answer, decoded_test) * 100:.2f}%")
print(f"Levenstein distace: {levenstein_distance(message_answer, decoded_test)}")
decoded_test

Char decoding accuracy: 99.57%
Levenstein distace: 1


'если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаъ'

Триграммы сработали лучше, чем биграммы. Сообщение полностью дешифровалось, без последней буквы))

#### 4-граммы


In [79]:
corpus_frequencies_fourgramm = get_frequencies(tokenized_corpus, n_gram=4)
reverse_mapping = generate_mcmc_reverse_mapping(message, alphabet_message, alphabet_ru, corpus_frequencies_fourgramm, n_gram=4, epochs=EPOCHS)

Epoch #1
Current log-likelihood: -2184.5921398183245
Current accepted proposals: 503


Epoch #2
Current log-likelihood: -2813.691432782589
Current accepted proposals: 1145


Epoch #3
Current log-likelihood: -2184.5921398183245
Current accepted proposals: 1600


Epoch #4
Current log-likelihood: -2184.5921398183245
Current accepted proposals: 2048


Epoch #5
Current log-likelihood: -2585.950645005328
Current accepted proposals: 2462


...

Epoch #96
Current log-likelihood: -2836.371172907099
Current accepted proposals: 68421


Epoch #97
Current log-likelihood: -2875.8852094433996
Current accepted proposals: 69131


Epoch #98
Current log-likelihood: -2836.4196129707593
Current accepted proposals: 69804


Epoch #99
Current log-likelihood: -2903.304492989464
Current accepted proposals: 70834


Epoch #100
Current log-likelihood: -2839.470850378946
Current accepted proposals: 71474
Best likelihood: -2184.5921398183245
Acceptance ratio: 0.071474


In [80]:
decoded_test = apply_mapping(message, reverse_mapping)
print(f"Char decoding accuracy: {char_accuracy(message_answer, decoded_test) * 100:.2f}%")
print(f"Levenstein distace: {levenstein_distance(message_answer, decoded_test)}")
decoded_test

Char decoding accuracy: 99.57%
Levenstein distace: 1


'если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещац'

Получились идентичные сообщения с высокой долей дешифровки.